### Funktionen zum Spiegeln

Wie zuvor erklärt, werden in diesem Notebook Stellungen gespiegelt, um die Effizienz der Berechnung zu erhöhen.
Der folgende Abschnitt erklärt die damit verbundende Funktionalität.

Die globale Variable SWAPS enthält für jede Spiegelungsart ein Dictionary. 
In diesem Dictionary liegt für jedes `Square` (Nummeriert von 0-63) eine Zuweisung, welches `Square` der gespiegelten Position entspricht.

So wird beispielsweise bei der vertikalen Spiegelung die Figuren der obersten Reihe mit der untersten Reihe getauscht.
Eine Zuweisung im Dictionary wäre beispielsweise: `0: 56`.   

In [ ]:
SWAPS = {
        "vertical" : {x : (x ^ 56) for x in range(64)},
        "horizontal" : {x : (x ^ 7) for x in range(64)},
        "rotate_right" : {x : ((((x >> 3) | (x << 3)) & 63) ^ 56) for x in range(64)},
        "rotate_180" : {x : (x ^ 63) for x in range(64)},
        "rotate_left" : {x: ((((x >> 3) | (x << 3)) & 63) ^ 7) for x in range(64)},
        "diagonal" : {x: (((x >> 3) | (x << 3)) & 63) for x in range(64)},
        "anti_diagonal" : {x: ((((x >> 3) | (x << 3)) & 63) ^ 63) for x in range(64)}
    }

Die Spiegelung wird von der Funktion `mirror_board()` für eine Stellung `board_int` durchgeführt.
Sie speichert das Bit, welches den Spieler am Zug repräsentiert, iteriert über alle Figuren im Integer und führt einen binären Tausch entsprechend der Zuweisungen im `mirror` Dictionary.
Da im Integer die Positionen einzelner Figuren kodiert sind, können Spiegelungen direkt durch Verändern des Integers durchgeführt werden. 

In [ ]:
# Mirroring mit 7 Bit Darstellung
def mirror_board(board, swap : dict):
    '''
    # Save turn
    result = board_int & 1
    # Get count of pieces saved in int
    n = len(piece_list)
    # Remove turn
    board_int = board_int >> 1
    
    for i in range(n):
        result |= mirror[board_int & 127] << 7 * i + 1
        board_int = board_int >> 7
    return result
    '''

    swapped_board = chess.Board(None)
    swapped_board.turn = board.turn

    for position, piece in board.piece_map().items():
        swapped_board.set_piece_at(swap[position], piece)
    return swapped_board


Die Funktion `mirror_all_directions()` erstellt für einen Integer (`board_int`) eine Menge aller Spiegelungen mit den Methoden, die in `SWAPS` aufgeführt sind.
Die Menge `result` enthält den original Integer `board_int` nicht.

In [ ]:
# Implementierung ohne ENUM:
def mirror_all_directions(board_int, piece_list):
    result = set()
    board = to_board(board_int, piece_list)
    for name, swap in SWAPS.items():
        result.add(to_integer(mirror_board(board, swap), piece_list))
    '''
    board = to_board(board_int, piece_list)
    result.add(to_integer(board.transform(chess.flip_vertical), piece_list))
    result.add(to_integer(board.transform(chess.flip_horizontal), piece_list))
    result.add(to_integer(board.transform(chess.flip_diagonal), piece_list))
    result.add(to_integer(board.transform(chess.flip_anti_diagonal), piece_list))
    '''
    return result


Die Funktion `get_all_mirror_variations()` berechnet für eine Menge von Stellungen in Integer-Repräsentation (`uniques_int`) die Menge der gespiegelten Variationen inklusive der Original stellungen (`result`).
Hierfür wird die Funktion `mirror_all_directions()` verwendet.

In [ ]:
def get_all_mirror_variations(uniques_int, piece_list):
    result = uniques_int.copy()
    for unique in uniques_int:
        result |= mirror_all_directions(unique, piece_list)
    return result


Soll eine zuvor berechnete Endspielsituation zur Auswertung verwendet werden, müssen alle $n$ gespiegelt werden.
Die Erstellung der vollständigen $S_n$ Mengen geschieht durch die Funktion `gen_all_integers()`.
Diese nimmt die bereits gespeicherten Mengen in ``s_n_integers`` und iteriert über diese.
Für jedes Brett der Menge werden die 7 Spiegelungen generiert.
Diese werden in einer Liste von Mengen, `result`, gespeichert.
Die Indizes der Mengen werden dabei nicht verändert. Die Liste `result` stellt den Rückgabewert der Funktion dar.

In [ ]:
def gen_all_integers(s_n_integers, piece_list):
    result = []
    for sequence in s_n_integers:
        int_set = set()
        for int_board in sequence:
            int_set.add(int_board)
            int_set |= mirror_all_directions(int_board, piece_list)
        result.append(int_set)
    return result
